In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 2. CSV 파일들 로드
df = pd.read_csv('./preprocessing/apllication_train_4.csv', encoding='utf-8-sig')

In [ ]:
# 세 비율의 분위수(quantile) 확인
metrics = ['대출대비소득비율','연금대비소득비율','대출대비연금비율']

for m in metrics:
    print(m)
    print(df[m].quantile([0.5, 0.75, 0.9, 0.95]))

대출대비소득비율
0.50    3.26510
0.75    5.15990
0.90    7.48750
0.95    9.15675
Name: 대출대비소득비율, dtype: float64
연금대비소득비율
0.50    0.1628
0.75    0.2291
0.90    0.3016
0.95    0.3547
Name: 연금대비소득비율, dtype: float64
대출대비연금비율
0.50    20.0000
0.75    27.1000
0.90    34.0631
0.95    34.5423
Name: 대출대비연금비율, dtype: float64


**1. 대출대비소득비율(LTI)**  

- 50% : **3.27**
    - 절반의 사람은 대출 금액이 연소득의 약 3.3배 이하
- 75% : **5.16**
    - 상위 25%는 연소득의 5배 이상 대출
- 90% : **7.49**
    - 상위 10%는 연소득의 7.5배 이상 대출
- 연소득 대비 대출 규모가 큰 사람(특히 5배 이상)은 위험군 후보.

---

**2. 연금대비소득비율 (DSR)**

- 중앙값 **0.16**
    - 절반의 사람은 소득의 16% 이하를 월 상환에 사용.

- 75% **0.229**
    - 상위 25%는 소득의 23% 이상을 월 상환에 사용.

- 90% **0.302**
    - 상위 10%는 30% 이상을 월 상환에 사용.
- DSR이 30%를 넘는 구간(상위 10%)은 상당히 상환 부담이 크다고 볼 수 있음.

---

**3. 대출대비연금비율 (상환 기간 지표)**
- 중앙값 20
    - 대출금 ÷ 월 상환액 = 20 → 대략 20개월(1.5년) 수준.

- 75% 27.1
    - 상위 25%는 27개월 이상 걸림.

- 90% 34.1
    - 상위 10%는 34개월 이상 걸림.

- 값이 크다는 건 상환 기간이 길다는 뜻 → 부채가 장기간 지속.

**각 지표별 점수**  
```
값 < 75% 분위수  : 0점  
75% ≤ 값 < 90%  : 1점  
값 ≥ 90%        : 2점  
```

**risk_level**  
```
총점 0~1점 → Low  
총점 2~3점 → Medium  
총점 4~6점 → High  
```

**해석**  
**High**   
: 세 지표 중 2개 이상이 상위 10% 이상  
**Medium**  
: 1~2개 지표가 상위 25% 이상  
**Low**  
: 대부분 지표가 중간 이하  

In [ ]:
# 분위수 기준선
q = {
    '대출대비소득비율': (5.15990, 7.48750),
    '연금대비소득비율': (0.2291, 0.3016),
    '대출대비연금비율': (27.1, 34.0631)
}

def score(val, q75, q90):
    if val >= q90:
        return 2
    elif val >= q75:
        return 1
    else:
        return 0

# risk_score 계산
df['risk_score'] = (
    df['대출대비소득비율'].apply(lambda x: score(x, *q['대출대비소득비율'])) +
    df['연금대비소득비율'].apply(lambda x: score(x, *q['연금대비소득비율'])) +
    df['대출대비연금비율'].apply(lambda x: score(x, *q['대출대비연금비율']))
)

# 등급화
df['risk_level'] = pd.cut(df['risk_score'],
                          bins=[-1, 1, 3, 6],
                          labels=['Low', 'Medium', 'High'])


In [11]:
print(df['risk_level'].value_counts())

risk_level
Low       217138
Medium     60275
High       30098
Name: count, dtype: int64


In [13]:
print(df.groupby('risk_level')['TARGET'].mean())

risk_level
Low       0.084002
Medium    0.073861
High      0.070868
Name: TARGET, dtype: float64


C:\Users\COTTA\AppData\Local\Temp\ipykernel_14284\3175451131.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('risk_level')['TARGET'].mean())


- 고위험으로 분류된 사람들이 상환 능력이 좋은 것 같음
- 고객군 자체가 무담보, 소액대출 중심이라서 위같은 방식은 영향력이 적은 것 같음
- 부채 부담 정도로 보는게 맞는 것 같음.
- `risk_level` -> `burden_level`

In [15]:
df.rename(columns={
    'risk_level': 'burden_level',
    'risk_score': 'burden_score'
}, inplace=True)

In [ ]:
df['AGE'] = (-df['DAYS_BIRTH'] / 365).astype(int)

# AGE_BIN
bins = [0, 29, 39, 49, 59, 69, 100]
labels = ['20대 이하', '30대', '40대', '50대', '60대', '70대 이상']
df['AGE_BIN'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=True, include_lowest=True)

In [42]:
# burden_level == High인 subset
df_high = df[df['burden_level'] == 'High']

# 직업별 인원수
print('-'*33)
print('직업별 인원수')
print(df_high['OCCUPATION_TYPE'].value_counts(normalize=True).round(4))
# print(df_high['OCCUPATION_TYPE'].value_counts())

# 연령대 구간별 분포
print('-'*33)
print('연령대 구간별 분포')
print(df_high['AGE_BIN'].value_counts(normalize=True).round(4))
# print(df_high['AGE_BIN'].value_counts())

# 평균 소득, 평균 대출 비교
print('-'*33)
print('평균 소득, 평균 대출 비교')
print(df.groupby('burden_level')[['AMT_INCOME_TOTAL', 'AMT_CREDIT']].mean().round(0))

---------------------------------
직업별 인원수
OCCUPATION_TYPE
Laborers                 0.2379
Sales staff              0.1702
Core staff               0.1539
Managers                 0.0703
Medicine staff           0.0613
Drivers                  0.0574
High skill tech staff    0.0538
Accountants              0.0509
Security staff           0.0380
Cooking staff            0.0371
Cleaning staff           0.0301
Private service staff    0.0097
Secretaries              0.0083
Low-skill Laborers       0.0077
Waiters/barmen staff     0.0058
HR staff                 0.0033
Realty agents            0.0022
IT staff                 0.0021
Name: proportion, dtype: float64
---------------------------------
연령대 구간별 분포
AGE_BIN
50대       0.2861
40대       0.2685
30대       0.2303
60대       0.1366
20대 이하    0.0785
70대 이상    0.0000
Name: proportion, dtype: float64
---------------------------------
평균 소득, 평균 대출 비교
              AMT_INCOME_TOTAL  AMT_CREDIT
burden_level                              
Low      

C:\Users\COTTA\AppData\Local\Temp\ipykernel_14284\1376394274.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('burden_level')[['AMT_INCOME_TOTAL', 'AMT_CREDIT']].mean().round(0))


In [43]:
# 1. 직업별 비율 (burden_level별로 그룹화)
occupation_dist = (
    df.groupby('burden_level')['OCCUPATION_TYPE']
    .value_counts(normalize=True)
    .rename('proportion')
    .reset_index()
)

# 2. 보기 좋게 pivot
pivot_table = occupation_dist.pivot(
    index='OCCUPATION_TYPE',
    columns='burden_level',
    values='proportion'
).fillna(0).round(3)

# 3. High burden 기준으로 정렬
pivot_table = pivot_table.sort_values(by='High', ascending=False)

# 4. 결과 확인
print(pivot_table)


burden_level             Low  Medium   High
OCCUPATION_TYPE                            
Laborers               0.269   0.243  0.238
Sales staff            0.149   0.155  0.170
Core staff             0.125   0.139  0.154
Managers               0.104   0.105  0.070
Medicine staff         0.037   0.045  0.061
Drivers                0.094   0.081  0.057
High skill tech staff  0.053   0.057  0.054
Accountants            0.044   0.052  0.051
Security staff         0.031   0.031  0.038
Cooking staff          0.027   0.029  0.037
Cleaning staff         0.021   0.021  0.030
Private service staff  0.013   0.013  0.010
Secretaries            0.006   0.006  0.008
Low-skill Laborers     0.011   0.008  0.008
Waiters/barmen staff   0.007   0.005  0.006
HR staff               0.003   0.003  0.003
Realty agents          0.004   0.003  0.002
IT staff               0.003   0.002  0.002


C:\Users\COTTA\AppData\Local\Temp\ipykernel_14284\1367836321.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('burden_level')['OCCUPATION_TYPE']


| 직업                 | Low → High 비율 변화 | 인사이트                                   |
| ------------------ | ---------------- | -------------------------------------- |
| **Laborers**       | 26.9% → 23.8%    | 비율 감소 (여전히 가장 많은 직군이지만 부담은 비례하지 않음)    |
| **Sales staff**    | 14.9% → 17.0%    | 증가 → **영업직 고부담군**                   |
| **Core staff**     | 12.5% → 15.4%    | 증가 → **사무직 내 부담 증가**                |
| **Medicine staff** | 3.7% → 6.1%      | 뚜렷한 증가 → **중소병원·개인 병원 종사자 고부담** 가능성 |
| **Cleaning staff** | 2.1% → 3.0%      | 증가 → **저숙련 서비스직 고부담 군**             |


##### 고부담 페르소나 (High burden 대표)
- 사무직/판매직/의료보조/청소 등
- 30~50대 중소득층 중심, 고정 소득은 있으나 대출이 큼
- 소득은 중간이지만 상환 부담이 구조적으로 큼
- 직군 예시: Sales staff, Core staff, Medicine staff, Cleaning staff

##### 저부담 페르소나 (Low burden 대표)
- 관리직, 고숙련직 (IT, 회계), 운전직 등
- 소득 대비 대출 규모 작고, 상환기간도 짧음
- 중장년층 우량 고객군

In [47]:
print(df.groupby('OCCUPATION_TYPE')['대출대비소득비율'].quantile([0.75, 0.9]))

OCCUPATION_TYPE            
Accountants            0.75    5.250000
                       0.90    7.471760
Cleaning staff         0.75    5.555600
                       0.90    8.022960
Cooking staff          0.75    5.555600
                       0.90    7.986700
Core staff             0.75    5.315800
                       0.90    7.628110
Drivers                0.75    4.625100
                       0.90    6.514500
HR staff               0.75    5.190350
                       0.90    7.766540
High skill tech staff  0.75    5.130100
                       0.90    7.200000
IT staff               0.75    4.538550
                       0.90    6.677800
Laborers               0.75    4.951075
                       0.90    7.072000
Low-skill Laborers     0.75    4.741300
                       0.90    6.845700
Managers               0.75    4.540300
                       0.90    6.526300
Medicine staff         0.75    5.833300
                       0.90    8.333300
Private serv

In [46]:
print(df.groupby('AGE_BIN')['연금대비소득비율'].mean())

AGE_BIN
20대 이하    0.168929
30대       0.176165
40대       0.180907
50대       0.188737
60대       0.192238
70대 이상         NaN
Name: 연금대비소득비율, dtype: float64


C:\Users\COTTA\AppData\Local\Temp\ipykernel_14284\751719490.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('AGE_BIN')['연금대비소득비율'].mean())
